In [1]:
import torch 
import torch.nn.functional as F 
import random 
import numpy as np 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# On choisit un modèle très léger (environ 500 Mo)
model_name = "facebook/opt-125m" 

print("Téléchargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Téléchargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(model_name)

# On déplace le modèle sur le GPU si disponible, sinon CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(f"Modèle chargé sur {device} !")

c:\Users\BONETTI\Desktop\3A\Intro to generative models\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Téléchargement du tokenizer...
Téléchargement du modèle...
Modèle chargé sur cpu !


In [4]:
import torch

def compute_log_likelihood(model, tokenizer, sequence):
    """Calcule la log-vraisemblance log(p(x)) d'une séquence."""
    inputs = tokenizer(sequence, return_tensors="pt").to(model.device)
    input_ids = inputs["input_ids"]
    
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        # Le modèle renvoie déjà la CrossEntropy (Negative Log Likelihood moyenne)
        # On la multiplie par le nombre de tokens pour avoir la somme des log-probs
        log_p_x = -outputs.loss.item() * (input_ids.shape[1] - 1)
        
    return log_p_x, inputs

In [8]:

def run_correction_tracker(model, tokenizer, initial_text, alpha=15.0, steps=10, block_size=15):
    current_text = initial_text
    # Calcul initial
    current_log_p, _ = compute_log_likelihood(model, tokenizer, current_text)
    
    history = []
    history.append({"step": 0, "text": current_text, "log_p": current_log_p, "status": "Initial"})

    print(f"Départ: {current_text} | Log P: {current_log_p:.2f}")

    for i in range(1, steps + 1):
        # 1. On transforme le texte en IDs pour manipuler les blocs
        input_ids = tokenizer.encode(current_text, return_tensors="pt").to(model.device)
        seq_len = input_ids.shape[1]

        # 2. Choisir un bloc à modifier (on évite le tout début)
        start_idx = random.randint(min(5, seq_len-1), max(5, seq_len - block_size - 1))
        prefix_ids = input_ids[:, :start_idx]

        # 3. Proposer un nouveau bloc (x')
        with torch.no_grad():
            new_block_ids = model.generate(
                prefix_ids, 
                max_new_tokens=block_size, 
                do_sample=True, 
                temperature=1.0, # On échantillonne normalement
                attention_mask=prefix_ids.ne(tokenizer.pad_token_id).long(),
                pad_token_id=tokenizer.eos_token_id
            )
        
        proposed_text = tokenizer.decode(new_block_ids[0], skip_special_tokens=True)
        #print(f"Phrase complète à cette étape : \n {proposed_text}")
        proposed_log_p, _ = compute_log_likelihood(model, tokenizer, proposed_text)

        # 4. Calcul du ratio Metropolis-Hastings
        # log(A) = alpha * (log_p_proposed - log_p_current)
        acceptance_log_ratio = alpha * (proposed_log_p - current_log_p)
        
        accepted = False
        if np.log(random.random()) < acceptance_log_ratio:
            current_text = proposed_text
            current_log_p = proposed_log_p
            accepted = True

        status = "ACCEPTÉ" if accepted else "REJETÉ"
        print(f"Étape {i}: {status} | Nouveau texte: {proposed_text[:50]}... | Log P: {proposed_log_p:.2f}")
        
        history.append({
            "step": i, 
            "text": proposed_text, 
            "log_p": proposed_log_p, 
            "status": status,
            "final_text_at_step": current_text
        })

    return history

# --- TEST ---
# On commence volontairement avec une phrase un peu bancale
prompt_initial = "The most important number in math is"
tracker_results = run_correction_tracker(model, tokenizer, prompt_initial, alpha=16, steps=20)

Départ: The most important number in math is | Log P: -26.93
Étape 1: REJETÉ | Nouveau texte: The most important number in the world, it was a 9... | Log P: -56.22
Étape 2: REJETÉ | Nouveau texte: The most important number of words: EASILY, EASILY... | Log P: -64.79
Étape 3: REJETÉ | Nouveau texte: The most important number would be 8, and the most... | Log P: -51.16
Étape 4: REJETÉ | Nouveau texte: The most important number of times I know the word... | Log P: -67.34
Étape 5: REJETÉ | Nouveau texte: The most important number to use is the "T" in the... | Log P: -59.06
Étape 6: REJETÉ | Nouveau texte: The most important number of people I know, and pr... | Log P: -53.93
Étape 7: REJETÉ | Nouveau texte: The most important number is the number of days a ... | Log P: -56.81
Étape 8: REJETÉ | Nouveau texte: The most important number is 6.  This isn't even a... | Log P: -57.98
Étape 9: REJETÉ | Nouveau texte: The most important number in any city in the world... | Log P: -60.26
Étape 10: RE

In [9]:
def compare_greedy_vs_power(model, tokenizer, prompt, alpha=15.0):
    # 1. Génération Greedy (Standard)
    greedy_output = model.generate(
        tokenizer(prompt, return_tensors="pt").input_ids.to(model.device),
        max_new_tokens=20,
        do_sample=False # Mode Greedy
    )
    greedy_text = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    greedy_log_p, _ = compute_log_likelihood(model, tokenizer, greedy_text)

    # 2. Génération Power Sampling (On prend le meilleur de 5 chaînes MCMC)
    power_results = []
    for i in range(5):
        # On lance un MCMC court
        trace = run_correction_tracker(model, tokenizer, prompt, alpha=alpha, steps=10, block_size=5)
        final_version = trace[-1]
        power_results.append(final_version)

    # Trier pour trouver le meilleur résultat Power Sampling
    best_power = max(power_results, key=lambda x: x['log_p'])

    print("\n--- RÉSULTATS ---")
    print(f"GREEDY: {greedy_text} | LogP: {greedy_log_p:.2f}")
    print(f"POWER : {best_power['final_text_at_step']} | LogP: {best_power['log_p']:.2f}")
    
    return greedy_log_p, best_power['log_p']

# Test
g_score, p_score = compare_greedy_vs_power(model, tokenizer, "The most important number in math is")

Départ: The most important number in math is | Log P: -26.93
Étape 1: REJETÉ | Nouveau texte: The most important number you can make a statement... | Log P: -34.69
Étape 2: REJETÉ | Nouveau texte: The most important number for all of us is... | Log P: -28.69
Étape 3: REJETÉ | Nouveau texte: The most important number of people in this game... | Log P: -28.63
Étape 4: REJETÉ | Nouveau texte: The most important number to remember to check is... | Log P: -30.03
Étape 5: ACCEPTÉ | Nouveau texte: The most important number of people in the world... | Log P: -24.12
Étape 6: REJETÉ | Nouveau texte: The most important number of things to know on... | Log P: -32.46
Étape 7: REJETÉ | Nouveau texte: The most important number to me is when you... | Log P: -27.64
Étape 8: REJETÉ | Nouveau texte: The most important number will be the "N... | Log P: -33.82
Étape 9: REJETÉ | Nouveau texte: The most important number of people my whole life... | Log P: -33.12
Étape 10: REJETÉ | Nouveau texte: The most imp